# Web Clawer 爬蟲 Google 評論

reference: https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/ 
https://medium.com/marketingdatascience/google-map-review-%E5%8B%95%E6%85%8B%E7%88%AC%E8%9F%B2-%E5%9B%9B-%E5%A6%82%E4%BD%95%E7%8D%B2%E5%8F%96%E5%BA%97%E5%AE%B6%E8%A9%95%E8%AB%96%E5%85%A7%E5%AE%B9-%E5%9C%96%E7%89%87%E4%BB%A5%E5%8F%8A%E7%AF%A9%E9%81%B8%E8%A9%95%E8%AB%96-%E9%99%84python%E7%A8%8B%E5%BC%8F%E7%A2%BC-33a06a671d13


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from bs4 import BeautifulSoup
import re
import time



## 1. 測試區

In [4]:
driver = webdriver.Chrome() #開啟虛擬的google Chrome 
#要記得把chrome_driver放在子目錄下

In [5]:
driver.get("https://www.google.com.tw/maps/@23.8759391,120.588669,8z?hl=zh-TW") #進入google map
element = driver.find_element_by_id("searchboxinput") #找到網頁中的 search box
element.send_keys("彼刻餐酒館") #填入資料
time.sleep(5) #等待網頁load
search_button = driver.find_element_by_class_name("searchbox-searchbutton") #找到網頁中搜尋的 button
search_button.click() #進入要搜尋的網頁
time.sleep(5) #等待網頁load
expand_button = driver.find_element_by_class_name("widget-pane-link") #找到網頁中評論的 button
expand_button.click() #


### 測試下滑頁面

In [6]:
pane = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')  #找到左邊的方框
driver.execute_script("arguments[0].scrollBy(0, 300)", pane) #向下滑(0到300)  

### 展開評論的網頁

In [7]:
expand_review_button =  driver.find_element_by_css_selector('[jsaction = "pane.review.expandReview"]')
expand_review_button.click()
driver.execute_script("arguments[0].scrollBy(0, 300)", pane) #向下滑(0到300)  

In [8]:
#如果有多筆資料，以第一筆的資料蒐集結果為主
driver.get("https://www.google.com.tw/maps/@23.8759391,120.588669,8z?hl=zh-TW") #進入google map
element = driver.find_element_by_id("searchboxinput") #找到網頁中的 search box
element.send_keys("麥當勞") #填入資料
time.sleep(5) #等待網頁load
search_button = driver.find_element_by_class_name("searchbox-searchbutton") #找到網頁中搜尋的 button
search_button.click() #進入要搜尋的網頁
time.sleep(5)
try : #尋找符合的第一筆資料
#     first_button = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[4]/div[1]/div[1]/div/a')
    first_button = driver.find_element_by_css_selector("[jsaction*='clickmod']")
    first_button.click()
except:
    print("except")


### 測試抓評論

In [ ]:
driver = webdriver.Chrome()

In [9]:
driver.get("https://www.google.com.tw/maps/@23.8759391,120.588669,8z?hl=zh-TW")
element = driver.find_element_by_id("searchboxinput") #找到網頁中的 search box
element.send_keys("彼刻餐酒館") #填入資料
search_button = driver.find_element_by_class_name("searchbox-searchbutton") #找到網頁中搜尋的 button
search_button.click() #
try: #如果有多個選項就選第一個
    first_button = driver.find_element_by_css_selector("[jsaction*='clickmod']")
    first_button.click()
except:
    print("Already find the result")
time.sleep(3)

expand_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME,("widget-pane-link"))) #找到網頁中評論的 button
    )
expand_button.click() #展開評論
time.sleep(3)
print("start rolling")

pane2 = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')  #找到左邊的方框
driver.execute_script("arguments[0].scrollBy(0, 900)", pane2) #向下滑(0到300)  

try: #如果評論可以展開，展開評論
    expand_review_button =  driver.find_element_by_css_selector('[jsaction = "pane.review.expandReview"]') 
    expand_review_button.click()
    driver.execute_script("arguments[0].scrollBy(0, 50)", pane) #向下滑(0到300)  
except:
    print("nothing can expand")
    
#找rating 跟 評論 放在Review中
get_star = driver.find_element(By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div[1]/div/div[3]/div[3]/div[1]/span[2]')
get_comment = driver.find_element(By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div[1]/div/div[3]/div[3]/div[2]/span[2]')


Review = dict({"Comment": [], "Rating":[]})

Review["Comment"].append(get_comment.text)
Review["Rating"].append(float(get_star.get_attribute('aria-label')[0:2]))


Already find the result
start rolling
nothing can expand


In [10]:
print(Review)


{'Comment': ['疫情下的彼刻：符合現階段的餐飲準則，隔板、距離、分盤。菜單特別挑選，防疫「午/晚餐」每人800/1200低消。\n\n三人點了四前菜（煙燻豬頭捲、海鮮蛋煎、炸鯪魚球、炭烤蔬菜），四主餐（海盜寬扁麵、蒜味辣義麵、伊比利豬、含雞腳的大雞腿）\n兩甜品（油醋橄欖油冰淇淋），一瓶葡萄汁。\n\n餐飲感受：\n1.為了符合疫情限制，餐廳空間可接來客數勢必減少，只好拉高低消價格，調整菜單，使能讓餐廳營運順利。\n2.客人為了達到低消，也就需要點得多些，尤其我們沒有點「酒」類，所以實在吃得真真飽。\n3.菜色、口味的品質很不錯，有別於一般義式餐廳，尤其是義麵Q彈有嚼勁，特別喜歡。\n4.服務部分特別一提：炸鯪魚球裡出現約莫1公分的魚細刺，店家表示有小小刺是正常但如此長度則確有不妥，因此感到抱歉並招待該道料理。關於此事，三人當中一人沒感覺就享受入腹，一人感到有刺但不以為意，咬一咬也入口了。唯有我吃東西不囫圇吞棗、細細品味才會發現有這樣長度的刺並取出來。不過，也因為有各式各樣的客人，所以店家必須謹慎料理每一次的餐點。\n5.疫情下的餐廳飲食體驗不同於以往，$$得多些、店家服務會好些、用餐環境無吵雜人聲、但隔板還是會影響用餐的愜意美好氛圍⋯⋯，期待下次更加愉悅的餐飲感受。'], 'Rating': [4.0]}


# 2. Class

In [20]:
class Google_Review():
    
    def __init__(self, place_name, driver):
        self.place_name = place_name
        self.driver = driver
        self.driver.get("https://www.google.com.tw/maps/@23.8759391,120.588669,8z?hl=zh-TW")
        element = self.driver.find_element_by_id("searchboxinput") #找到網頁中的 search box
        element.send_keys(place_name) #填入資料
        search_button = self.driver.find_element_by_class_name("searchbox-searchbutton") #找到網頁中搜尋的 button
        search_button.click() #
        time.sleep(5)
        
        try : #尋找符合的第一筆資料
            print('first element')
            
            self.first_button = self.driver.find_element_by_css_selector("[jsaction*='clickmod']")
            self.first_button.click()
        except:
            print("except")
        
        
         
        expand_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME,("widget-pane-link"))) #找到網頁中評論的 button
            )
        expand_button.click() #
        time.sleep(3)
        print("start rolling")

        self.pane = self.driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')  #找到左邊的方框
        driver.execute_script("arguments[0].scrollBy(0, 100)", self.pane) #向下滑(0到300) 

        try: #如果評論可以展開，展開評論
            self.expand_review_button =  self.driver.find_element_by_css_selector('[jsaction = "pane.review.expandReview"]') 
            self.expand_review_button.click()
            self.driver.execute_script("arguments[0].scrollBy(0, 100)", self.pane) #向下滑(0到300)  
        except:
            print("nothing can expand")
            
        self.present_div_count = 1
        self.XPATH_star = '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div['+ str(self.present_div_count) +']/div/div[3]/div[3]/div[1]/span[2]'
        self.XPATH_comment = '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div['+ str(self.present_div_count) +']/div/div[3]/div[3]/div[2]/span[2]'
    
        get_star = self.driver.find_element(By.XPATH, self.XPATH_star)
        get_comment = self.driver.find_element(By.XPATH, self.XPATH_comment)
        
        self.Review = dict({"Comment": [], "Rating":[]})
        self.Review["Comment"].append(get_comment.text)
        self.Review["Rating"].append(float(get_star.get_attribute('aria-label')[0:2]))

        
    def get_next_review(self):
        time.sleep(1)
        
        try: #如果評論可以展開，展開評論
            self.expand_review_button =  self.driver.find_element_by_css_selector('[jsaction = "pane.review.expandReview"]') 
            self.expand_review_button.click()
            self.driver.execute_script("arguments[0].scrollBy(0, 150)", self.pane) #向下滑(0到300)
            time.sleep(1)
        except:
            print("nothing can expand")
            
        #更新要尋找的評論的XPATH
        self.present_div_count += 3 #加3是經驗法則
        self.XPATH_star = '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div['+ str(self.present_div_count) +']/div/div[3]/div[3]/div[1]/span[2]'
        self.XPATH_comment = '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[9]/div['+ str(self.present_div_count) +']/div/div[3]/div[3]/div[2]/span[2]'
    
        print("test next review")
        try:
            get_star = self.driver.find_element(By.XPATH, self.XPATH_star)
            get_comment = self.driver.find_element(By.XPATH, self.XPATH_comment)
            self.Review["Comment"].append(get_comment.text)
            self.Review["Rating"].append(float(get_star.get_attribute('aria-label')[0:2]))
        except: #如果不能抓，就滑到頁面底部
            self.driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight)", self.pane) 
            time.sleep(5)
            try: #滑到頁面底部後再次測試抓評論
                get_star = self.driver.find_element(By.XPATH, self.XPATH_star)
                get_comment = self.driver.find_element(By.XPATH, self.XPATH_comment)
                self.Review["Comment"].append(get_comment.text)
                self.Review["Rating"].append(float(get_star.get_attribute('aria-label')[0:2]))
            except: #如果還是不能抓應該就是沒有評論了
                print("Already Get the Last Comment")
#             get_next_review(self)
            
    def get_review(self, number): #選擇你要拿幾個 review
        self.number = number
        for _ in range(0,self.number):
            try:
                self.get_next_review()
            except:
                print("can't get reviews")
                break
                
            
        

    

In [21]:
driver = webdriver.Chrome()

In [22]:
try_class = Google_Review("摩斯漢堡", driver)

first element
start rolling


In [25]:
#取出你想要的評論數
try_class.get_review(20)

test next review
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review


In [26]:
print(try_class.Review)

{'Comment': ['服務不錯，可能因為疫情的關係比較好打電話過去點餐。三個套餐外帶大概15分鐘內處理好，跟路程算一致。\n珍珠堡蠻有飽足感的，不過玉米濃湯比較粉粉的，口感沒有特別好，雞塊很特別，吃起來很好咬，皮比較薄。', '除了不好停車以外，沒有其他可以挑剔的地方，服務也很棒', '這間分店生意還不錯，所以有時候會等比較久，店面有點復古感，但貼皮的地面已經有點斑駁、損壞，有時候髒東西都卡在地板的隙縫中，不是很乾淨的感覺，希望有機會能翻修。\n內用的座位還算多，孩童與學生常在此出沒，一樓聲音有點兒吵雜，二樓比較安靜。\n店內的音樂固定都是同樣的旋律，常來的客人聽得有點兒膩，可以輪播新的歌曲較有新意。\n服務還算不錯，老店員會認常客，有時候很令人感覺貼心、溫馨。', '每次如果餐期來買都要等20分鐘以上，趕時間不適合點的速食店，是目前吃過最慢的mos漢堡店', '紅茶跟糖水一樣 一點茶香味都沒有 全台中就屬於這間 最糟 可不可以學一下豐原店 或是 台灣大道中科', '愛吃紅豆派及海鮮米漢堡，歷久不衰！', '在網路上看到職缺 可以固定班\n去到那邊面試的人也沒準備履歷表\n還直接叫我去櫃檯那邊拿\n然後還要輪班 ，啊網路上是在寫好玩的？\n還問說輪班跟出來找工作有需要問家裡的人嗎？\n請問找工作關家裡什麼事情？\n\n真的浪費我時間！！', '服務員態度很差 才點完兩份套餐也不問還有沒有需要的直接幫我結帳\n啊我還要點其他餐啊^ ^', '其中一個服務人員態度很差，點餐問題回話顯得不耐煩，拿號碼牌等候用命令大聲的口氣，而另外兩位服務不錯也客氣（戴眼鏡），請公司重視員工素質！明明東西很好吃卻被員工的態度影響光顧的意願，可惜….', '在崇德路上很好找但是不好停車，裡面有兩台自助點餐機，點完餐後記得拿號碼牌取餐，現在好像也有接受線上點餐，很方便', '使用店內點餐機點餐後，那了號碼牌因不知要結帳後，點餐流程很有問題，等了20分鐘後櫃檯才問我取餐了沒，比我晚進來的都一個結帳取餐都離開了，我等了20分鐘才有人來訊問，櫃檯還很不客氣的說“你現在點才會開始做“，等我離開剛好30分鐘，流程及服務不佳！', '服務不錯，餐點協助送到位子，服務態度很好。', '最喜歡海洋珍珠堡， 不過價格真的算貴就是了', '防疫措施優良 服務好 餐點佳', '店員對餐飲優惠不熟，最好先準備好

In [14]:
#得到下一則評論
for _ in range(0,20):
    try_class.get_next_review()

nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review
test next review
test next review
test next review
test next review
test next review
test next review
test next review
nothing can expand
test next review
nothing can expand
test next review
nothing can expand
test next review


In [68]:
print(try_class.Review['Comment'])
print(try_class.Review['Rating'])


['難得一片沙灘可供游泳及灘釣又可欣賞美麗夕陽西下的美景', '漁光島是位於安平區緊鄰安平漁港與商港的一個小島，面積約400多公頃，島上一半以上的面積都是森林，近年不時舉行文創市集，在自然風光外也增添許多文化氣息。逛完漁光島可到白河萬里長城，絕美的自然景觀，東巷與關子嶺碧雲寺對望，西向可以觀賞高速公路至海岸景觀，無論日出日落都充滿夢幻之美，高山廣闊的視野非常美好，空氣清新令人舒暢愉悅，日落後更可以欣賞360度夜景與星空星軌，交通方便離高速公路與白河蓮花賞景區只需15分鐘，喜歡旅遊攝影的朋友局不能錯過的新景點。', '很早以前是秋茂園黃秋茂先生創建.懷念以前秋茂園的大小事.烤肉玩沙.一旁沙地很多那種會挖洞的(沙豬)都用頭髮綁螞蟻或小昆蟲就可以釣出來.周圍有可以控窯的地方.還有馬場可以騎迷你馬.擺沙畫的攤販(那時候超級流行沙畫)還有賣蚵仔煎茶葉蛋彈珠汽水店家.可惜之後改建成漁光島.雖然比較美觀也比較乾淨.但是已經沒有以前那種純樸單純的時光回憶.現在安平辦活動這邊都是人擠人.可以說外地人必來的景點吧.要來建議騎摩托車好停車.漁光路與健康路三段叉口兩側都是畫紅線要注意.還有晚上別來因為路燈有點暗加上距離遠.視線不佳.', '人好多QQ\n但環境蠻乾淨的！感覺有用心在維護，感謝辛苦的維護人員\n附近的小廟也可以洗手洗腳上廁所，很棒^-^來這坐在石頭上發呆看夕陽很不錯～', '附近為商港海域，有玩風帆、衝浪板、快艇還有戲水，但沒見到救生員，建議做好配套管理，提高整體遊客的停留時間。', '黃昏時分去，看海和夕陽，愜意時光。', '之前就聽說漁光島很有名，因此特別挑黃昏的時間點慕名而來，來島上需要過一座橋，過橋之後就會看到滿滿的車流跟人山人海的人流，下橋之後映入眼簾的是兩側停滿機車再往前走會有各式各樣的攤販的旁邊，我找好停車位之後就趕緊前往海灘了。\n\n一踩上沙灘看到橘黃色的夕陽映在海岸線上非常感動，於是馬上拿起自己的單眼開始進行拍攝，雖然說這次準備不足沒有帶可以踩水的裝備跟衣服，但是單純淨淨的待在黃昏時分的沙灘上看海看夕陽的感覺還是非常涼爽舒服。海岸上有許多遊客在拍照，建議有想來的人可以穿拖鞋來踩水，應該會非常舒服有趣喔！', '景色開闊，很美的地方，可以踏浪，孩子們可以玩沙', '漁光島藝術季，真的讓人有種放鬆的感覺，文青又悠閒', '快六點才到\n來看夕陽～', '台南的